# Column Sorting Challenge

## Semantic value solution
In this Notebook we are mapping the values from origin dataset to target columns based on semantic values.

The solution is divided into the following steps:

  - Load OPENAI API key from .env file
  - Load dataset from remote CSV files into dataframes
  - Create a list of target columns for each dataset
  - Load gpt-3.5-turbo model as an agent using Langchain
  - Define a simple function which maps the values in the rows to the target columns and print the results

To execute this notebook, adjust the environment variables in the .env file and run the cells below.


In [1]:
!pip install -U -q langchain langchain-community langchain-openai langchainhub openai pandas python-dotenv

In [2]:
import pandas as pd
from dotenv import load_dotenv
from langchain import hub
from langchain.agents import AgentExecutor, create_json_chat_agent
from langchain_openai import ChatOpenAI

In [3]:
load_dotenv('.env')
vehicles_df = pd.read_csv('https://raw.githubusercontent.com/bsantanna/notebooks/master/challenges/column_sorting/data/vehicles.csv')
animals_df = pd.read_csv('https://raw.githubusercontent.com/bsantanna/notebooks/master/challenges/column_sorting/data/animals.csv')

vehicles_target_columns=["road", "air", "water", "rails"]
animals_target_columns=["mammal", "bird", "fish", "reptile"] 

In [4]:
# initializes JSON agent
# Reference: https://python.langchain.com/v0.1/docs/modules/agents/agent_types/json_agent/
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
prompt = hub.pull("hwchase17/react-chat-json")
tools = []
agent = create_json_chat_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=False,
    handle_parsing_errors=True
)

In [5]:
def map_columns(item_df, target_columns):
    print(item_df)
    print(f"----\ntarget columns: {target_columns}\n----")
    for index, row in item_df.iterrows():
        row_string = ','.join(row.to_string(index=False).split('\n'))
        response = agent_executor.invoke({"input": f"Map items from row {row_string} to columns {target_columns}"})
        print(response['output'])

In [6]:
map_columns(vehicles_df, vehicles_target_columns)

   vehicle_1 vehicle_2 vehicle_3 vehicle_4
0        car     train     plane      boat
1  submarine       bus     metro    rocket
2  mine_cart      ship     truck   bicycle
----
target columns: ['road', 'air', 'water', 'rails']
----
car: road, train: rails, plane: air, boat: water
submarine: water, bus: road, metro: rails, rocket: air
mine_cart: rails, ship: water, truck: road, bicycle: road


In [8]:
map_columns(animals_df, animals_target_columns)

   animal_1 animal_2 animal_3   animal_4
0     shark      cat   lizard     parrot
1  elephant    eagle    whale  crocodile
2   penguin   monkey    snake       tuna
----
target columns: ['mammal', 'bird', 'fish', 'reptile']
----
shark: fish, cat: mammal, lizard: reptile, parrot: bird
elephant: mammal, eagle: bird, whale: fish, crocodile: reptile
penguin: bird, monkey: mammal, snake: reptile, tuna: fish


## Conclusion
This is a simple example using Pandas and Langchain to map the values in the rows to the target columns.

A more elaborate approach would be to use combination of [Langchain CSV toolkit](https://python.langchain.com/v0.1/docs/integrations/toolkits/csv/) and [Langchain Requests Tool](https://python.langchain.com/v0.1/docs/integrations/tools/requests/) to read remote CSV files and map the values to the target columns.
